In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import uuid
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')

In [ ]:
# path to your CSV file
file_path = '/content/drive/MyDrive/St/mocked_financial_data.csv'

try:
  df = pd.read_csv(file_path)
  # print(df.head())
except FileNotFoundError:
  print(f"File not found at: {file_path}")


File not found at: /content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/mocked_financial_data.csv


In [ ]:
df.shape

In [ ]:
unique_id = df.IdentifierValue
df.drop(['IdentifierValue'],axis=1,inplace=True)

In [ ]:
df=df.replace('missing',np.nan)
df=df.replace('na',np.nan)

In [ ]:
df.info()

In [ ]:
# prompt: find date time features in df

import pandas as pd
# Find columns with datetime features
date_features = []
for col in df.columns:
  if pd.api.types.is_datetime64_any_dtype(df[col]):
    date_features.append(col)

print("Datetime columns found:", datetime_cols)


In [ ]:
fig = df.isna().sum().sort_values().plot(kind = 'barh', figsize = (7, 12))
plt.title('Percentage of Missing Values Per Column in Data Set', fontdict={'size':15})
drop_cols=[]
for p in fig.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/df.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    fig.annotate(percentage,(x,y))
    if (p.get_width()/df.shape[0])*100>90:
      drop_cols.append(fig.get_yticklabels()[int(y)].get_text())
plt.show()

In [ ]:
# drop the columns with missing value percentage greater than 90% which are saves in drop_cols
df=df.drop(drop_cols,axis=1)
# save drop_cols to csv file
pd.DataFrame(drop_cols).to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/missing_value_columns.csv', index=False)

In [ ]:
for feature in date_features:

  df[feature].fillna(method='ffill', inplace=True)
  # Convert to datetime objects
  df[feature] = pd.to_datetime(df[feature])

  # Extract features
  df[feature + '_year'] = df[feature].dt.year
  df[feature + '_month'] = df[feature].dt.month
  df[feature + '_day'] = df[feature].dt.day
  df[feature + '_dayofweek'] = df[feature].dt.dayofweek
  df[feature + '_quarter'] = df[feature].dt.quarter
  df[feature + '_is_weekend'] = (df[feature].dt.dayofweek // 5 == 1).astype(int)
  df.drop([feature],axis=1,inplcae=True)


In [ ]:
# check for variance
numerical_features = [feature for feature in df.columns if df[feature].dtype!='O' and df[feature].dtype!='datetime64[ns]']
variance = df[numerical_features].var(skipna=True)
print(variance)
# drop features with low variance
low_variance_features = variance[variance < 0.001].index
df = df.drop(low_variance_features, axis=1)
#save features to csv file
pd.DataFrame(low_variance_features).to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/low_variance_features.csv', index=False)

In [ ]:
# drop features with single unique value
single_value_features = [feature for feature in df.columns if df[feature].nunique(dropna=True) == 1]
df = df.drop(single_value_features, axis=1)
# save dropped features to csv file
pd.DataFrame(single_value_features).to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/single_value_features.csv', index=False)

In [ ]:
# save all the dropped features in single csv file
dropped_features = pd.concat([pd.DataFrame(low_variance_features), pd.DataFrame(single_value_features)], axis=0)
# save the fdropped features in csv file
dropped_features.to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/dropped_features.csv', index=False)

In [ ]:
# find all numerical features
numerical_features = [feature for feature in df.columns if df[feature].dtype!='O' and df[feature].dtype!='datetime64[ns]']

# calculate mean and median for each numerical feature
mean_values = df[numerical_features].mean(skipna=True)
median_values = df[numerical_features].median(skipna=True)

# create a dataframe with the features, mean and median
feature_stats = pd.DataFrame({'Feature': numerical_features, 'Mean': mean_values, 'Median': median_values})

# write to csv file
feature_stats.to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/feature_stats.csv', index=False)

# find all categorical features
categorical_features = df.select_dtypes(include=['object']).columns

# find the high frequency category for each categorical feature
high_frequency_categories = df[categorical_features].apply(lambda x: x.mode()[0])

# create a dataframe with features and their high fequency category
high_frequency_df = pd.DataFrame({'Feature': high_frequency_categories.index, 'High Frequency Category': high_frequency_categories.values})

# write to csv
high_frequency_df.to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/high_frequency_categories.csv', index=False)

In [ ]:
[df[numerical_features].isnull().mean()>0][0].index

In [ ]:
# load stats file
stats = pd.read_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/feature_stats.csv')

# create a dictionary for quick lookup
mean_dict = stats.set_index('Feature')['Mean'].to_dict()
median_dict = stats.set_index('Feature')['Median'].to_dict()

# find numerical features with missing values
numerical_features_with_missing_values = [df[numerical_features].isnull().mean()>0][0].index

# fill all the missing values of numerical features based on outlier, in case of outlier fill with median, else mean
for feature in numerical_features_with_missing_values:
  if df[feature].isnull().sum()>0:
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    has_outliers = (df[feature] < lower_bound) | (df[feature] > upper_bound)
    if has_outliers.any():
      df[feature].fillna(median_dict[feature], inplace=True)
    else:
      df[feature].fillna(mean_dict[feature], inplace=True)

    df[feature].fillna(mean_dict[feature], inplace=True)

#load the high frequency category data from the csv file
high_frequency_categories = pd.read_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/high_frequency_categories.csv')

# create dictionary for quick lookup
high_frequency_dict = high_frequency_categories.set_index('Feature')['High Frequency Category'].to_dict()

#find categorical features with missing values
categorical_features_with_missing_values = [df[categorical_features].isnull().mean()>0][0].index

# fill missing values of categorical features with high frequency category
for feature in categorical_features_with_missing_values:
  if feature in high_frequency_dict:
    df[feature].fillna(high_frequency_dict[feature], inplace=True)


In [ ]:
df.columns[df.isnull().sum()>0]

In [ ]:
df.head()

In [ ]:
categorical_features = ['IdentifierType', 'AccountingIntent']

In [ ]:
df[categorical_features].nunique()

In [ ]:
df['IdentifierType'].value_counts()

In [ ]:
df['AccountingIntent'].value_counts()

In [ ]:
df[numerical_features].nunique()

In [ ]:
#encoding

# write a fucntion for one hot encoding
def one_hot_encode(df, columns):
  for column in columns:
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
  return df

# write a fucntion for frequency encoding
def frequency_encode(df, columns):
  for column in columns:
    frequencies = df[column].value_counts(normalize=True)/(len(df)*0.01)
    df[column + '_freq'] = df[column].map(frequencies)
    df = df.drop(column, axis=1)
  return df

#write a function to rank categories based on frequency, highest frequency get 1, followed by 2,3,4
def frequency_ranking_encode(df, columns):
  for column in columns:
    frequencies = df[column].value_counts(normalize=True)
    frequencies={category: rank+1 for rank, category in enumerate(frequencies.index)}
    df[column + '_freq'] = df[column].map(frequencies)
    df = df.drop(column, axis=1)
  return df


In [ ]:
# find all numerical features
numerical_features = [feature for feature in df.columns if df[feature].dtype!='O' and df[feature].dtype!='datetime64[ns]']

# find discrete numerical features with nunique <10
discrete_numerical_features = [feature for feature in numerical_features if df[feature].nunique() < 10]

# find continuous numerical features
continuous_numerical_features = [feature for feature in numerical_features if feature not in discrete_numerical_features]

# convert discreate numerical features into string
df[discrete_numerical_features] = df[discrete_numerical_features].astype(str)

# find all categorical features
categorical_features = [feature for feature in df.columns if df[feature].dtype=='O']

df = frequency_ranking_encode(df, categorical_features)

# find all numerical features -updated
numerical_features = [feature for feature in df.columns if df[feature].dtype!='O' and df[feature].dtype!='datetime64[ns]']

In [ ]:
df.head()

In [ ]:
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Find highly correlated columns (e.g., correlation > 0.9)
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
  for j in range(i):
    if abs(correlation_matrix.iloc[i, j]) > 0.9:
      colname_i = correlation_matrix.columns[i]
      colname_j = correlation_matrix.columns[j]
      highly_correlated_features.add((colname_i, colname_j))

print("Highly correlated features:")
for feature_pair in highly_correlated_features:
  print(feature_pair)



In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
pca = PCA(n_components=0.9)
df_pca = pd.DataFrame(pca.fit_transform(df))

In [ ]:
# calculate feature contribution for each principal component
feature_contributions = pd.DataFrame(pca.components_,columns=df.columns)
# plot feature contribution
for i in range(len(feature_contributions)):
  plt.figure(figsize=(10,5))
  plt.bar(feature_contributions.columns,feature_contributions.iloc[i])
  plt.title('Feature Contribution for Principal Component {}'.format(i+1))
  plt.xticks(rotation=90)
  plt.show()

In [ ]:
# calculate explained variance and plot scree plot
explained_variance = pca.explained_variance_ratio_
plt.figure(figsize=(10,5))
plt.plot(range(1,len(explained_variance)+1),explained_variance,'o-')

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples

def plot_clustering(df,labels,title):
  fig = plt.figure(figsize=(8,6))
  ax = fig.add_subplot(111, projection='3d')
  unique_labels = np.unique(labels)
  for label in unique_labels:
    cluster_data = df[labels == label]
    ax.scatter(cluster_data.iloc[:,0],cluster_data.iloc[:,1],cluster_data.iloc[:,2],label=label)
  ax.set_title(title)
  ax.set_xlabel('PC1')
  ax.set_ylabel('PC2')
  ax.set_zlabel('PC3')
  ax.legend()
  plt.show()

def calculate_silhouette_scores(df,labels):
  unique_labels = np.unique(labels)
  avg_silhouette_scores = {}
  if len(unique_labels)>1:
    silhouette_scores = silhouette_samples(df,labels)
    cluster_silhouette_scores={}
    for label in unique_labels:
      cluster_indices = np.where(labels == label)[0]
      cluster_silhouette_scores[label] = silhouette_scores[cluster_indices]
    for label,score in cluster_silhouette_scores.items():
      avg_score = score.mean()
      print(f"Cluster {label}: Average Silhouette Score = {avg_score:.2f}")
      avg_silhouette_scores[label] = avg_score
      print(score)
      print()

    print("the average silhouette score is",silhouette_score(df,labels))
    return avg_silhouette_scores
  else:
    print("No clusters found")
    return None

In [ ]:
from sklearn.cluster import DBSCAN

# eps_values = np.arange(1,10,1)
# silhouette_scores = []
# for eps in eps_values:
#   dbscan = DBSCAN(eps=eps,min_samples=5)
#   labels = dbscan.fit_predict(df_pca)
#   unique_labels = np.unique(labels)
#   if len(unique_labels)>1:
#     silhouette_scores.append(silhouette_score(df_pca,labels))
#   else:
#     silhouette_scores.append(0)
# best_eps = eps_values[np.argmax(silhouette_scores)]
# print("Best eps:",best_eps)

best_eps= 8

DBSCAN = DBSCAN(eps=best_eps,min_samples=5)
labels = DBSCAN.fit_predict(df_pca)
plot_clustering(df_pca,labels,"DBSCAN Clustering")
average_silhoutte_scores=calculate_silhouette_scores(df_pca,labels)

In [ ]:
average_silhoutte_scores

In [ ]:
# prompt: add unique_id series to the dbscan labels

import pandas as pd
# Assuming 'labels' is the output from your DBSCAN clustering
labels_with_unique_id = pd.DataFrame({'unique_id': unique_id, 'cluster_label': labels})

# Now 'labels_with_unique_id' contains both the original unique identifier and the cluster label assigned by DBSCAN
print(labels_with_unique_id.head())


In [ ]:
labels_with_unique_id.cluster_label.value_counts()

In [ ]:
if average_silhoutte_scores:
  cluster_labels_below_threshold = [
      cluster_label for cluster_label, avg_score in average_silhoutte_scores.items()
      if avg_score < 0.7
  ]

  # Filter the DataFrame to include only rows with cluster labels below the threshold
  filtered_df = labels_with_unique_id[
      labels_with_unique_id.cluster_label.isin(cluster_labels_below_threshold)
  ]

  print(
      "Cluster labels with average silhouette score less than 0.7:",
      filtered_df.cluster_label.unique(),
  )
  print(filtered_df)
else:
  print("No clusters found or silhouette scores not calculated.")


In [ ]:
df_copy = pd.read_csv(file_path)

In [ ]:
# Get the unique identifiers from the filtered DataFrame
identifier_values_to_retrieve = filtered_df['unique_id'].tolist()

# Use these identifiers to retrieve datapoints from df_copy
retrieved_datapoints = df_copy[df_copy['IdentifierValue'].isin(identifier_values_to_retrieve)]

retrieved_datapoints

# write to csv
retrieved_datapoints.to_csv('/content/drive/MyDrive/CodeCrafters_WF/Model Input Data/Data/retrieved_datapoints.csv', index=False)